### **필요한 라이브러리 및 모듈 임포트**

- **`os`**: 운영체제 관련 기능을 제공하는 기본 Python 라이브러리로, 환경 변수를 가져오거나 설정할 때 사용합니다.
- **`Chroma`**: LangChain에서 제공하는 **벡터 데이터베이스**입니다. 벡터 검색을 사용하여 텍스트 임베딩을 관리하고 유사도 검색을 수행할 수 있습니다.
- **`OpenAIEmbeddings`**: OpenAI의 임베딩을 사용하는 클래스입니다. 텍스트 데이터를 벡터로 변환하는 데 사용됩니다.
- **`ChatOpenAI`**: OpenAI의 GPT 모델을 사용하기 위한 LangChain 클래스입니다. 대화를 기반으로 한 언어 모델을 사용하여 요청을 처리합니다.
- **`ChatPromptTemplate`**: 프롬프트 템플릿을 생성하여 다양한 입력을 처리하고 모델에게 제공할 수 있도록 돕습니다.
- **`RecursiveCharacterTextSplitter`**: LangChain에서 제공하는 텍스트 분할 도구로, 긴 텍스트를 일정한 크기로 나누기 위해 사용됩니다.
- **`LLMChain`**: LangChain에서 **언어 모델 체인**을 관리하는 도구로, 프롬프트와 모델을 연결하여 체계적으로 작업을 처리합니다.
- **`TextLoader`**: 텍스트 파일을 로드하는 클래스입니다. 파일을 읽고 문서로 변환하는 기능을 제공합니다.
- **`Document`**: LangChain에서 사용하는 문서 클래스입니다. 각 문서에는 텍스트 내용과 메타데이터가 포함될 수 있습니다.

### **OpenAI API 키 설정**
```python
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
```
- **`os.environ["OPENAI_API_KEY"]`**: OpenAI API에 접근하기 위해 API 키를 설정하는 코드입니다. 환경 변수(`OPENAI_API_KEY`)에 API 키를 저장하고 이를 프로그램에서 사용할 수 있도록 설정합니다.
  - `os.environ`은 환경 변수를 관리하는 Python의 기본 제공 객체입니다.
  
- **`os.getenv("OPENAI_API_KEY")`**: 환경 변수에서 **OpenAI API 키**를 가져옵니다. 이 값은 미리 설정된 환경 변수에서 가져오며, 프로그램이 OpenAI API에 접근할 수 있게 해줍니다.

### **전체 흐름**
이 코드의 전체적인 흐름은 LangChain을 사용하여 다음과 같은 작업을 수행하기 위한 준비 작업입니다:
1. **벡터 저장소 생성**: `Chroma`를 사용하여 텍스트 임베딩을 벡터로 저장하고, 유사도 검색을 가능하게 합니다.
2. **OpenAI 임베딩**: `OpenAIEmbeddings` 클래스는 텍스트 데이터를 벡터로 변환하기 위해 OpenAI의 임베딩 모델을 사용합니다.
3. **텍스트 분할**: `RecursiveCharacterTextSplitter`는 긴 텍스트를 1000자 크기 등의 청크로 나누는 역할을 합니다. 이 작업은 대규모 텍스트를 처리할 때 효율적으로 번역 및 처리할 수 있도록 도와줍니다.
4. **번역 작업**: `ChatOpenAI`와 `LLMChain`을 사용하여 OpenAI GPT 모델에 텍스트를 보내 번역을 수행합니다.
5. **문서 처리**: `TextLoader`는 텍스트 파일을 읽어 `Document` 객체로 변환합니다. 이 객체는 나중에 벡터 스토어에 저장되거나 검색 작업에 사용됩니다.

### **주요 사용 예시**
- **텍스트 번역**: OpenAI API를 사용하여 텍스트를 영어에서 한국어로 번역할 수 있습니다.
- **벡터 데이터베이스 관리**: `Chroma` 벡터 스토어를 사용하여 임베딩된 텍스트 데이터를 저장하고, 유사도 검색을 통해 관련 텍스트를 빠르게 검색할 수 있습니다.
- **텍스트 청크 분할**: 긴 텍스트 파일을 처리할 때 텍스트를 일정한 크기로 나눠 모델이 처리할 수 있는 형태로 분리합니다.
- **프롬프트 템플릿**: 번역 작업을 위한 일관된 프롬프트를 생성하고 OpenAI 모델에 전달하여 효율적인 작업을 수행합니다.

[Introduction | 🦜️🔗 LangChain](https://python.langchain.com/v0.2/docs/introduction/)

In [ ]:
# !pip install -Uq langchain chromadb

In [ ]:
import os
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import LLMChain
from langchain.document_loaders import TextLoader
from langchain.docstore.document import Document

# OpenAI API 키 설정
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [ ]:
# 첫 번째 텍스트 파일 찾기 함수
def get_first_txt_file(directory='.'):
    txt_files = [f for f in os.listdir(directory) if f.endswith('yp6WuHFhYCo_subtitle.txt')]
    txt_files.sort()  # 파일명 알파벳 순으로 정렬
    return txt_files[0] if txt_files else None

# 벡터 스토어 생성 함수
def create_vector_store():
    embeddings = OpenAIEmbeddings()
    return Chroma("langchain_store", embeddings)

# LangChain 기반 번역 함수
def translate_text(text):
    llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
    prompt = ChatPromptTemplate.from_template(
        "You are a translator. Translate the following text from English to Korean: {text}"
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    return chain.run(text=text)

# 번역 결과 파일로 저장
def save_translations_to_file(vector_store, output_file="langchain_translations.txt"):
    with open(output_file, "w", encoding="utf-8") as f:
        results = vector_store.similarity_search("", k=len(vector_store.get()))
        for doc in results:
            f.write(f"Original: {doc.page_content}\n")
            f.write(f"Translation: {doc.metadata['translation']}\n")
            f.write("---\n")
    print(f"번역 결과가 {output_file}에 저장되었습니다.")

# 번역 결과 검색 함수
def search_translations(vector_store, query, k=5):
    results = vector_store.similarity_search(query, k=k)
    print(f"'{query}'와 유사한 상위 {k}개 번역 결과:")
    for i, doc in enumerate(results, 1):
        print(f"{i}. Original: {doc.page_content}")
        print(f"   Translation: {doc.metadata['translation']}")
        print("---")

# 번역 통계 확인
def get_translation_statistics(vector_store):
    results = vector_store.similarity_search("", k=len(vector_store.get()))
    total_chars_original = sum(len(doc.page_content) for doc in results)
    total_chars_translated = sum(len(doc.metadata['translation']) for doc in results)
    avg_chars_original = total_chars_original / len(results)
    avg_chars_translated = total_chars_translated / len(results)
    
    print(f"총 번역된 청크 수: {len(results)}")
    print(f"원본 총 문자 수: {total_chars_original}")
    print(f"번역 총 문자 수: {total_chars_translated}")
    print(f"원본 평균 문자 수: {avg_chars_original:.2f}")
    print(f"번역 평균 문자 수: {avg_chars_translated:.2f}")

# 메인 함수
def main():
    first_txt_file = get_first_txt_file()
    if not first_txt_file:
        print("No .txt files found in the current directory.")
        return

    # 텍스트 파일 로드
    loader = TextLoader(first_txt_file, encoding='utf-8')
    documents = loader.load()
    
    # LangChain의 RecursiveCharacterTextSplitter 사용
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000, chunk_overlap=0
    )
    chunks = text_splitter.split_documents(documents)
    
    print(f"파일 '{first_txt_file}'에서 총 {len(chunks)}개의 청크가 생성되었습니다.")

    # 벡터 스토어 생성
    vector_store = create_vector_store()

    print("번역을 시작합니다...")
    for i, chunk in enumerate(chunks):
        translated = translate_text(chunk.page_content)
        doc = Document(page_content=chunk.page_content, metadata={"translation": translated})
        vector_store.add_documents([doc])
        print(f"청크 {i+1}/{len(chunks)} 번역 완료")

    print("번역이 완료되었습니다.")

    # 결과 출력 (예시로 처음 5개 문서)
    results = vector_store.similarity_search("", k=5)
    for doc in results:
        print(f"Original: {doc.page_content}")
        print(f"Translation: {doc.metadata['translation']}")
        print("---")

if __name__ == "__main__":
    main()

In [ ]:
vector_store = create_vector_store()

# 번역 결과 파일로 저장
save_translations_to_file(vector_store)

# 특정 쿼리로 유사한 번역 검색
search_translations(vector_store, "PSF", k=3)

# 번역 통계 출력
get_translation_statistics(vector_store)

In [ ]:
results = vector_store.similarity_search("Python", k=len(vector_store.get()))

In [ ]:
results